In [1]:
import tensorflow as tf
import numpy as np
from components.positional import add_timing_signal_nd
import pathlib
import os

## Load Images

In [2]:
data_dir = pathlib.Path("data/small")

In [3]:
imgs_objs = list(data_dir.glob("*.png"))

In [4]:
imgs_objs

[PosixPath('data/small/8.png'),
 PosixPath('data/small/9.png'),
 PosixPath('data/small/4.png'),
 PosixPath('data/small/5.png'),
 PosixPath('data/small/7.png'),
 PosixPath('data/small/6.png'),
 PosixPath('data/small/2.png'),
 PosixPath('data/small/3.png'),
 PosixPath('data/small/1.png'),
 PosixPath('data/small/0.png')]

In [5]:
os.fspath(imgs_objs[0])

'data/small/8.png'

In [6]:
imgs_paths = [os.fspath(img) for img in imgs_objs]

In [7]:
imgs_paths = sorted(set(imgs_paths))

In [8]:
imgs_paths

['data/small/0.png',
 'data/small/1.png',
 'data/small/2.png',
 'data/small/3.png',
 'data/small/4.png',
 'data/small/5.png',
 'data/small/6.png',
 'data/small/7.png',
 'data/small/8.png',
 'data/small/9.png']

In [9]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img)
    return img

In [10]:
image_dataset = tf.data.Dataset.from_tensor_slices(imgs_paths)

In [11]:
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(3)

In [12]:
val_dataset = image_dataset.take(3)

## CNN

In [13]:
embedding_dim = 80

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', input_shape=(50, 120, 3)))
model.add(tf.keras.layers.MaxPool2D(padding='same'))

model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(padding='same'))

model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))

model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))

model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))

model.add(tf.keras.layers.Conv2D(512,  2, strides=(2, 4), activation='relu'))

model.add(tf.keras.layers.Conv2D(512,  3, activation='relu'))

model.add(tf.keras.layers.Lambda(add_timing_signal_nd))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(embedding_dim))

In [19]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none'), metrics=['accuracy'])



In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 120, 64)       1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 60, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 30, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 30, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 30, 256)       590080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 30, 512)       1

In [21]:
model.fit(image_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10


ValueError: in user code:

    /Users/frankpalma/Projects/img2latex/env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/frankpalma/Projects/img2latex/env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/frankpalma/Projects/img2latex/env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/frankpalma/Projects/img2latex/env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/frankpalma/Projects/img2latex/env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        retur

In [23]:
for img in image_dataset:
    print(img.shape)

(3, 50, 120, 3)
(3, 50, 120, 3)
(3, 50, 120, 3)
(1, 50, 120, 3)
